In [2]:
# Import calibration functions
# Run this to reimport without having to restart the whole notebook
import radial_velocity_functions
import importlib
importlib.reload(radial_velocity_functions)
from radial_velocity_functions import *

In [3]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt

matplotlib.rcParams["text.usetex"] = True
matplotlib.rcParams["font.family"] = "serif"
matplotlib.rcParams["font.size"] = 10 
matplotlib.rcParams['figure.dpi'] = 150

In [7]:


filename = r"expres_tp/LFC_200907.1063.fits"

# Load data
hdu1 = fits.open(filename)

data = hdu1[1].data.copy()

results = []
# for order in range(0, len(data)):
# for order in range(40, 76):
order = 44

data_spec       = data['spectrum'][order]
data_spec_err   = data['uncertainty'][order]
data_wavel      = data['wavelength'][order]

# Find peaks
peak_info = func_find_peaks(data_spec, 11, 0.15)
peak_locs = peak_info[0]

# Create data slices around each peak
peak_index_ranges = get_peak_index_ranges(peak_locs)

# Fit peak in each data slice
peak_fits = fit_peaks(data_spec, data_spec_err, peak_index_ranges)

# Get list of true wavelengths
wavel_true = get_true_wavel(data_wavel, peak_locs)

results.append([order, peak_fits, wavel_true, data_wavel[peak_locs]])


# results = np.asarray(results, dtype=object)


# CALIB: 

cols = ['A', "A_err", "mu", 'mu_err', 'sigma', 'sigma_err', 'c', 'c_err', "chi2", 'ndof', 'converged', 'index_start', 'index_end', 'p-value', 'P', 'P_err']
peak_fits_df = pd.DataFrame(peak_fits, columns=cols)
chi2_mask = (peak_fits_df.chi2 < 100)
peak_fits = peak_fits[chi2_mask]
wavel_true = wavel_true[chi2_mask]

# Skip if out of peaks... 
if len(peak_fits) < 1:
    print("yoooo no peaks ")

# Take every other element
x_interp = peak_fits[:,2][::2]
y_interp = wavel_true[::2][:, 0]

f2 = interp1d(x_interp, y_interp, kind='cubic') 

# Residuals for all x and y
x_remain = peak_fits[:,2][1::2] # Take every other element starting at index 1 (so we get the other ones), and skip last item
y_remain = wavel_true[1::2][:, 0]
x_remain = x_remain[1:-1]  # first and last data points don't work well — exclude them
y_remain = y_remain[1:-1]

y_predicted = f2(x_remain)

# res = y_predicted - y # residuals
c = 299792458 # m/s
res = (y_predicted - y_remain)/y_remain * c


238